In [2]:
def su3_tensor3(p, q):
    """Tensor (p,q) with (1,0). Return list of resulting (p',q')."""
    results = []
    results.append((p + 1, q))  # add box to first row
    if p - 1 >= 0:
        results.append((p - 1, q + 1))  # add box to second row
    if q - 1 >= 0:
        results.append((p, q - 1))  # add box to third row (in sense of lowering q)
    return results

def num_boxes(p, q):
    """Total boxes (for SU(3), ignoring 3-column equivalence)."""
    return p + 2 * q

def generate_su3_irreps(nmax):
    irreps = {}  # (p,q) -> {'code': int, 'parents': set()}
    frontier = []

    # --- Base: purely symmetric and antisymmetric irreps ---
    for p in range(1, nmax + 1):
        if num_boxes(p, 0) <= nmax:
            irreps[(p, 0)] = {'code': 1, 'parents': set()}
            frontier.append((p, 0))
    for q in range(1, nmax + 1):
        if num_boxes(0, q) <= nmax:
            irreps[(0, q)] = {'code': 1, 'parents': set()}
            frontier.append((0, q))

    current_code = 1
    while frontier:
        new_frontier = []
        for (p, q) in frontier:
            for child in su3_tensor3(p, q):
                boxes = num_boxes(*child)
                if boxes > nmax:
                    continue
                if child not in irreps:
                    irreps[child] = {'code': current_code + 1, 'parents': {(p, q)}}
                    new_frontier.append(child)
                else:
                    irreps[child]['parents'].add((p, q))
                    irreps[child]['code'] = min(
                        irreps[child]['code'], current_code + 1
                    )
        frontier = new_frontier
        current_code += 1
        if not frontier:
            break

    result_list = sorted(
        [
            (data["code"], p, q, num_boxes(p, q), sorted(data["parents"]))
            for (p, q), data in irreps.items()
        ],
        key=lambda x: (x[0], x[3], x[1], x[2]),
    )
    return result_list



nmax = 10
reps = generate_su3_irreps(nmax)
print(f"SU(3) irreps up to {nmax} boxes:")
for code, p, q, boxes, parents in reps:
    print(f"code={code:2d}  (p,q)=({p},{q})  boxes={boxes:2d}  parents={parents}")


SU(3) irreps up to 10 boxes:
code= 1  (p,q)=(1,0)  boxes= 1  parents=[(0, 0), (1, 1)]
code= 1  (p,q)=(0,1)  boxes= 2  parents=[(0, 2), (1, 0)]
code= 1  (p,q)=(2,0)  boxes= 2  parents=[(1, 0), (2, 1)]
code= 1  (p,q)=(3,0)  boxes= 3  parents=[(2, 0), (3, 1)]
code= 1  (p,q)=(0,2)  boxes= 4  parents=[(0, 3), (1, 1)]
code= 1  (p,q)=(4,0)  boxes= 4  parents=[(3, 0), (4, 1)]
code= 1  (p,q)=(5,0)  boxes= 5  parents=[(4, 0), (5, 1)]
code= 1  (p,q)=(0,3)  boxes= 6  parents=[(0, 4), (1, 2)]
code= 1  (p,q)=(6,0)  boxes= 6  parents=[(5, 0), (6, 1)]
code= 1  (p,q)=(7,0)  boxes= 7  parents=[(6, 0), (7, 1)]
code= 1  (p,q)=(0,4)  boxes= 8  parents=[(0, 5), (1, 3)]
code= 1  (p,q)=(8,0)  boxes= 8  parents=[(7, 0), (8, 1)]
code= 1  (p,q)=(9,0)  boxes= 9  parents=[(8, 0)]
code= 1  (p,q)=(0,5)  boxes=10  parents=[(1, 4)]
code= 1  (p,q)=(10,0)  boxes=10  parents=[(9, 0)]
code= 2  (p,q)=(0,0)  boxes= 0  parents=[(0, 1)]
code= 2  (p,q)=(1,1)  boxes= 3  parents=[(0, 1), (1, 2), (2, 0)]
code= 2  (p,q)=(2,1)  box

In [7]:
from collections import defaultdict


def tensor_with_fundamental(diagram):
    """
    Tensor a Young diagram with one fundamental (add 1 box).
    diagram: tuple of 3 numbers (r1,r2,r3) with r1>=r2>=r3>=0
    Returns list of resulting diagrams (still sorted r1>=r2>=r3)
    """
    r1,r2,r3 = diagram
    candidates = [
        (r1+1, r2, r3),
        (r1, r2+1, r3),
        (r1, r2, r3+1)
    ]
    # Keep only valid non-increasing rows
    children = []
    for c in candidates:
        sorted_c = tuple(sorted(c, reverse=True))
        children.append(sorted_c)
    return children

def all_young_diagrams(n):
    """
    Generate all valid SU(3) Young diagrams (3-row tuples) with up to n boxes.
    Each diagram satisfies r1 >= r2 >= r3 >= 0 and r1+r2+r3 <= n.
    Returns a list of tuples (r1,r2,r3).
    0 boxes is excluded.
    """
    diagrams = []
    for r1 in range(1, n+1):
        for r2 in range(r1+1):
            for r3 in range(r2+1):
                if r1 + r2 + r3 <= n:
                    diagrams.append((r1,r2,r3))
    return diagrams

def build_young_1new_rule(nmax=5, verbose=True):
    """
    Iteratively build Young diagrams using the 1-new-unknown rule:
    - Only process a parent if its tensor product yields 0 or 1 unknown children.
    - Otherwise, skip the parent for now and come back later.
    """
    known = set()
    
    # Seeds: purely symmetric and antisymmetric diagrams
    seeds = []
    for p in range(1, nmax+1):
        diag = (p,0,0)
        known.add(diag)
        seeds.append(diag)
    
    known.add((1,1,0)) # antisymmetric 2-box
    seeds.append((1,1,0))
    known.add((1,1,1)) # antisymmetric 3-box (trivial)
    seeds.append((1,1,1))

    
    # Parents pending processing
    pending = set(seeds)

    print("known:", sorted(list(known), key=lambda x: (sum(x), x)))

    step = 0
    while pending:
        progress = False  # track if we added any new diagram this iteration
        for parent in sorted(pending, key=lambda x: (sum(x), x)):
            if sum(parent) >= nmax:
                pending.remove(parent)
                continue

            raw_children = tensor_with_fundamental(parent)
            children = list({tuple(c) for c in raw_children})  # deduplicate

            unknown_children = [c for c in children if c not in known]

            if len(unknown_children) <= 1:
                step += 1
                # Add unknown children if any
                for ch in unknown_children:
                    known.add(ch)
                    pending.add(ch)

                # Parent is processed
                pending.remove(parent)
                progress = True

                if verbose:
                    print(f"Step {step}: parent={parent}, children={children}, new={unknown_children}")
                # Only process one parent per outer iteration to respect 1-new rule
                break

        if not progress:
            # No parent could be processed under 1-new-unknown rule
            print("No further progress possible under 1-new rule.")
            print(f"Remaining parents: {pending}")
            print("Detailed remaining parents info:")
            for parent in sorted(pending, key=lambda x: (sum(x), x)):
                raw_children = tensor_with_fundamental(parent)
                children = list({tuple(c) for c in raw_children})
                unknown_children = [c for c in children if c not in known]
                print(f"parent={parent}, children={children}, new={unknown_children}")
            break
    return known


if __name__ == "__main__":
    nmax = 6
    known = build_young_1new_rule(nmax)
    print("\nFinal known Young diagrams:")
    for k in sorted(list(known), key=lambda x: (sum(x), x)):
        print(f"diagram={k}, boxes={sum(k)}")

    print("all possible diagrams up to nmax:")
    all_diagrams = all_young_diagrams(nmax)
    for d in all_diagrams:
        print(f"diagram={d}, boxes={sum(d)}")

    if set(known) == set(all_diagrams):
        print("Known diagrams match all possible diagrams!")
    else:
        print("Known diagrams do not match all possible diagrams!")
        print(f"Missing diagrams: {set(all_diagrams) - set(known)}")


known: [(1, 0, 0), (1, 1, 0), (2, 0, 0), (1, 1, 1), (3, 0, 0), (4, 0, 0), (5, 0, 0), (6, 0, 0)]
Step 1: parent=(1, 0, 0), children=[(1, 1, 0), (2, 0, 0)], new=[]
Step 2: parent=(1, 1, 0), children=[(2, 1, 0), (1, 1, 1)], new=[(2, 1, 0)]
Step 3: parent=(2, 0, 0), children=[(3, 0, 0), (2, 1, 0)], new=[]
Step 4: parent=(1, 1, 1), children=[(2, 1, 1)], new=[(2, 1, 1)]
Step 5: parent=(3, 0, 0), children=[(3, 1, 0), (4, 0, 0)], new=[(3, 1, 0)]
Step 6: parent=(2, 1, 0), children=[(2, 1, 1), (3, 1, 0), (2, 2, 0)], new=[(2, 2, 0)]
Step 7: parent=(4, 0, 0), children=[(4, 1, 0), (5, 0, 0)], new=[(4, 1, 0)]
Step 8: parent=(5, 0, 0), children=[(5, 1, 0), (6, 0, 0)], new=[(5, 1, 0)]
No further progress possible under 1-new rule.
Remaining parents: {(2, 1, 1), (3, 1, 0), (4, 1, 0), (2, 2, 0)}
Detailed remaining parents info:
parent=(2, 1, 1), children=[(3, 1, 1), (2, 2, 1)], new=[(3, 1, 1), (2, 2, 1)]
parent=(2, 2, 0), children=[(2, 2, 1), (3, 2, 0)], new=[(2, 2, 1), (3, 2, 0)]
parent=(3, 1, 0), chil

In [ ]:
# 1new allowed each time 
def tensor_with_fundamental_dynkin(pq):
    """(p,q) ⊗ (1,0) -> possible children in Dynkin labels (p+1,q),(p-1,q+1),(p,q-1)."""
    p, q = pq
    candidates = [(p + 1, q), (p - 1, q + 1), (p, q - 1)]
    return [c for c in candidates if c[0] >= 0 and c[1] >= 0]

def boxes_from_dynkin(pq):
    """Number of boxes (after removing full 3-columns): p + 2*q"""
    p, q = pq
    return p + 2 * q

def build_young_1new_rule_dynkin_limited(nmax=6, verbose=True):
    """
    Build irreps in Dynkin (p,q) using 1-new-unknown rule, but only for irreps
    with p + 2*q <= nmax (i.e., truly coming from <= nmax boxes).
    """
    known = set()
    seeds = []

    # Seeds: pure symmetric (p,0) and pure antisymmetric (0,q) but only if boxes <= nmax
    for p in range(0, nmax + 1):
        if boxes_from_dynkin((p,0)) <= nmax:
            known.add((p, 0))
            seeds.append((p, 0))
    
    known.add((0, 1)) # antisymmetric 2-box
    seeds.append((0, 1))

    pending = set(seeds)
    step = 0

    if verbose:
        print("Initial known irreps:", sorted(known, key=lambda x:(boxes_from_dynkin(x), x)))

    while pending:
        progress = False
        # deterministic order: increasing box count then (p,q)
        for parent in sorted(pending, key=lambda x:(boxes_from_dynkin(x), x)):
            parent_boxes = boxes_from_dynkin(parent)
            # don't tensor parents that already have maximum boxes
            if parent_boxes >= nmax:
                pending.remove(parent)
                continue

            # generate children but only keep those whose box count <= nmax
            raw_children = tensor_with_fundamental_dynkin(parent)
            children = [c for c in raw_children if boxes_from_dynkin(c) <= nmax]

            # deduplicate children (set) and keep deterministic order
            children = sorted(set(children), key=lambda x:(boxes_from_dynkin(x), x))

            unknown_children = [c for c in children if c not in known]

            if len(unknown_children) <= 1:
                step += 1
                for ch in unknown_children:
                    known.add(ch)
                    pending.add(ch)
                pending.remove(parent)
                progress = True
                if verbose:
                    print(f"Step {step}: parent={parent}, children={children}, new={unknown_children}")
                break

        if not progress:
            if verbose:
                print("\nNo further progress possible under 1-new rule.")
                print(f"Remaining parents: {sorted(pending, key=lambda x:(boxes_from_dynkin(x), x))}")
                print("Detailed remaining parents info:")
                for parent in sorted(pending, key=lambda x:(boxes_from_dynkin(x), x)):
                    raw_children = tensor_with_fundamental_dynkin(parent)
                    children = [c for c in raw_children if boxes_from_dynkin(c) <= nmax]
                    unknown_children = [c for c in children if c not in known]
                    print(f"parent={parent}, children={children}, new={unknown_children}")
            break

    return known

if __name__ == "__main__":
    nmax = 6
    known = build_young_1new_rule_dynkin_limited(nmax, verbose=True)
    print("\nFinal known irreps (p,q) with boxcount <= nmax:")
    for k in sorted(known, key=lambda x:(boxes_from_dynkin(x), x)):
        print(f"{k}  boxes={boxes_from_dynkin(k)}")


Initial known irreps: [(0, 0), (1, 0), (0, 1), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0)]
Step 1: parent=(0, 0), children=[(1, 0)], new=[]
Step 2: parent=(1, 0), children=[(0, 1), (2, 0)], new=[]
Step 3: parent=(0, 1), children=[(0, 0), (1, 1)], new=[(1, 1)]
Step 4: parent=(2, 0), children=[(1, 1), (3, 0)], new=[]
Step 5: parent=(3, 0), children=[(2, 1), (4, 0)], new=[(2, 1)]
Step 6: parent=(1, 1), children=[(1, 0), (0, 2), (2, 1)], new=[(0, 2)]
Step 7: parent=(0, 2), children=[(0, 1), (1, 2)], new=[(1, 2)]
Step 8: parent=(2, 1), children=[(2, 0), (1, 2), (3, 1)], new=[(3, 1)]
Step 9: parent=(4, 0), children=[(3, 1), (5, 0)], new=[]
Step 10: parent=(5, 0), children=[(4, 1), (6, 0)], new=[(4, 1)]
Step 11: parent=(3, 1), children=[(3, 0), (2, 2), (4, 1)], new=[(2, 2)]
Step 12: parent=(1, 2), children=[(1, 1), (0, 3), (2, 2)], new=[(0, 3)]

No further progress possible under 1-new rule.
Remaining parents: []
Detailed remaining parents info:

Final known irreps (p,q) with boxcount <= nmax:
(0

In [13]:
# --- consistency check ---
# all possible valid (p,q) up to nmax boxes
all_possible = [(p, q) for p in range(nmax + 1)
                for q in range(nmax + 1)
                if p >= 0 and q >= 0 and p + 2*q <= nmax]

missing = set(all_possible) - set(known)
extra = set(known) - set(all_possible)

print("\nAll possible irreps with ≤ nmax boxes:")
for pq in sorted(all_possible, key=lambda x: (x[0] + 2*x[1], x)):
    print(f"{pq}  boxes={pq[0] + 2*pq[1]}")

if not missing and not extra:
    print(f"\n✅ Known irreps exactly match all {len(all_possible)} possible ones up to {nmax} boxes!")
else:
    if missing:
        print(f"\n⚠️ Missing irreps ({len(missing)}): {sorted(missing)}")
    if extra:
        print(f"⚠️ Extra irreps ({len(extra)}): {sorted(extra)}")


All possible irreps with ≤ nmax boxes:
(0, 0)  boxes=0
(1, 0)  boxes=1
(0, 1)  boxes=2
(2, 0)  boxes=2
(1, 1)  boxes=3
(3, 0)  boxes=3
(0, 2)  boxes=4
(2, 1)  boxes=4
(4, 0)  boxes=4
(1, 2)  boxes=5
(3, 1)  boxes=5
(5, 0)  boxes=5
(0, 3)  boxes=6
(2, 2)  boxes=6
(4, 1)  boxes=6
(6, 0)  boxes=6
(1, 3)  boxes=7
(3, 2)  boxes=7
(5, 1)  boxes=7
(7, 0)  boxes=7
(0, 4)  boxes=8
(2, 3)  boxes=8
(4, 2)  boxes=8
(6, 1)  boxes=8
(8, 0)  boxes=8
(1, 4)  boxes=9
(3, 3)  boxes=9
(5, 2)  boxes=9
(7, 1)  boxes=9
(9, 0)  boxes=9
(0, 5)  boxes=10
(2, 4)  boxes=10
(4, 3)  boxes=10
(6, 2)  boxes=10
(8, 1)  boxes=10
(10, 0)  boxes=10
(1, 5)  boxes=11
(3, 4)  boxes=11
(5, 3)  boxes=11
(7, 2)  boxes=11
(9, 1)  boxes=11
(11, 0)  boxes=11
(0, 6)  boxes=12
(2, 5)  boxes=12
(4, 4)  boxes=12
(6, 3)  boxes=12
(8, 2)  boxes=12
(10, 1)  boxes=12
(12, 0)  boxes=12
(1, 6)  boxes=13
(3, 5)  boxes=13
(5, 4)  boxes=13
(7, 3)  boxes=13
(9, 2)  boxes=13
(11, 1)  boxes=13
(13, 0)  boxes=13
(0, 7)  boxes=14
(2, 6)  boxes=1